In [1]:
import pandas as pd
import pyodbc
import numpy as np
from pandas import ExcelWriter
import datetime

In [2]:
fecha = datetime.date.today()

In [3]:
a=1

In [4]:
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+
                      ';DATABASE='+database+
                      ';UID='+username+
                      ';PWD='+ password) 

In [5]:
query = '''
select c.*,p.*,d.fecha_alta,d.id_sc2_corrida corrida_aux, d.stock,d.precio,d.precio_original,d.tipo_vigente TIPO,d.origen
from (
    select distinct id_sc2_corrida from sc2_detalle with (nolock)
    ) c
cross join 
    (
    select distinct id_sc2_producto, talle, color,tipo_vigente from sc2_detalle with (nolock)
    ) p
left join sc2_detalle d with (nolock) 
on d.id_sc2_corrida  = c.id_sc2_corrida  and d.id_sc2_producto = p.id_sc2_producto  and d.talle = p.talle and d.color = p.color and p.tipo_vigente = d.tipo_vigente
'''
query = pd.read_sql(query,cnxn)

In [6]:
if a==2:
    
    for i in query['id_sc2_corrida'].unique():
        print(i)
        print(len(query[query['id_sc2_corrida'] == i]))
        #474123

In [7]:

#ordenar df por campos
query.sort_values(['id_sc2_corrida','id_sc2_producto','talle','color','tipo_vigente'],inplace=True)

In [8]:
corridas_fechas = '''select distinct fecha_alta,id_sc2_corrida from sc2_detalle order by fecha_alta asc'''
corridas_fechas = pd.read_sql(corridas_fechas,cnxn)

corridas_fechas['timestamp'] = 0
#recorro corridas_fechas en reverse
for index,row in corridas_fechas.sort_values('fecha_alta',ascending=False).iterrows():
    #para no comparar el primero, creo.
    if index-1 == -1:
        break
    #resta del mayor al menor    
    corridas_fechas.loc[index,'timestamp'] = corridas_fechas.iloc[index]['fecha_alta'] - corridas_fechas.iloc[index-1]['fecha_alta']
#lleno los NaN con 0 para poder operar.
corridas_fechas.fillna(0,inplace=True)
#Regex para sacar valores numericos del campo 'timestamp' 
corridas_fechas['delta_dias'] = corridas_fechas['timestamp'].astype(str).str.extract(r"([\d]+)", expand=False).astype(int)
#corridas_fechas['delta_dias'] = [int(str(tmstmp)[0]) for tmstmp in corridas_fechas['timestamp']]
delta_dias = corridas_fechas['delta_dias'].sum()

In [9]:
corrida = '''select max(id_sc2_corrida) from sc2_detalle'''
HASTA_CORRIDA = pd.read_sql(corrida,cnxn)
HASTA_CORRIDA = int(HASTA_CORRIDA[""][0])

In [10]:
#parametros [DESDE, HASTA ]
master_fechas = corridas_fechas.copy()
DESDE_CORRIDA = 1
#HASTA_CORRIDA = 6

In [11]:
corridas_fechas = corridas_fechas[corridas_fechas[corridas_fechas['id_sc2_corrida'] == DESDE_CORRIDA].index[0]:
               HASTA_CORRIDA]

In [12]:
corridas_fechas.reset_index(drop=True,inplace=True)

In [13]:
corridas_fechas['mes'] = corridas_fechas['fecha_alta'].apply(lambda x : x.month)

In [14]:
MAX_CORRIDA = corridas_fechas['id_sc2_corrida'].max()

In [15]:
def reducir(df):
    if (df.corrida_aux.isna().sum() == len(df)-1
        and not np.isnan(df.corrida_aux.iloc[-1])):
            
        return df.dropna(how="any")
    else:
        return df

g = query.groupby(['id_sc2_producto', 'talle', 'color', 'tipo_vigente'])
result = g.apply(reducir).reset_index(drop=True)

In [16]:
quitar_de_analisis = []
for i in g:
    
    if (i[1].corrida_aux.isna().sum() == len(i[1])-1
        and not np.isnan(i[1].corrida_aux.iloc[-1])):
        quitar_de_analisis.append(i[0])

In [17]:
len(quitar_de_analisis)

12807

In [18]:
df_quitar = pd.DataFrame(quitar_de_analisis,columns=['id_sc2_producto','talle','color','tipo_vigente'])

In [19]:
notIsin = query[~query.set_index(['id_sc2_producto','talle','color','tipo_vigente']).index.isin(df_quitar.set_index(['id_sc2_producto','talle','color','tipo_vigente']).index)]

In [20]:
notIsin[
    (notIsin['id_sc2_producto']=='blouse171103173')&
    (notIsin['talle']=='L')&
    (notIsin['color']=='Pastel')&
    (notIsin['tipo_vigente']=='BLUSA')
]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta,corrida_aux,stock,precio,precio_original,TIPO,origen


In [21]:
query_copiado = query.copy()


In [22]:
len(query_copiado) == len(query)

True

In [23]:
if a ==2:
    display(corridas_fechas)

In [24]:
#query[(query['id_sc2_producto'] == 'RSWE161116103') & (query['talle']=='L')&(query['color']=='Multicolor')]

#resultado[(resultado['id_sc2_producto'] == 'RSWE161116103') & (resultado['talle']=='L')&(resultado['color']=='Multicolor')]

#PROBANDO[(PROBANDO['id_sc2_producto'] == 'RSWE161116103') & (PROBANDO['talle']=='L')&(PROBANDO['color']=='Multicolor')]

In [25]:
QUERY = pd.merge(notIsin,corridas_fechas,on='id_sc2_corrida',how='inner')

In [26]:
if a==2:
    display(QUERY.head())

In [27]:
if a ==2 :
    
    display(QUERY.groupby(['id_sc2_producto','talle','color','tipo_vigente','mes']).first())

In [28]:
#VER MINIMOS
minimos = corridas_fechas.groupby('mes').size().reset_index(name='cantidad')
minimos = minimos[minimos['cantidad']<2]

In [29]:
#OBSERVAR MINIMOS Y CREAR UNA LISTA CON LOS MINIMOS, PARA EVITAR PROBLEMAS 
minimos

,mes,cantidad


In [30]:
minimos['mes'].unique()

array([], dtype=int64)

In [31]:
query = QUERY.copy()

In [32]:
#ASIGNAR 1 SI STOCK TIENE 999

query['high_stock'] = 0
query.loc[query['stock'] == 999,'high_stock'] = 1
#**** Me falta ver si falta en una corrida *****
#ASIGNAR 1 si falta en una corrida

query['falta_corrida'] = 0
query.loc[query['corrida_aux'].isnull(),'falta_corrida'] = 1


In [33]:
for index,row in corridas_fechas.iterrows():
    
    if row['id_sc2_corrida'] +1 > len(corridas_fechas):
        break
        
    #CORRIDA ANTERIOR
    
    corrida_anterior = query[(query['id_sc2_corrida']== row['id_sc2_corrida'])&(query['mes'] == row['mes'])]
    corrida_anterior.set_index(['id_sc2_producto','talle','tipo_vigente','color'], inplace=True)
    corrida_anterior.reset_index(inplace=True)
    
#    if (corrida_anterior.corrida_aux.isna().sum() == len(corrida_anterior)-1
#        and not np.isnan(corrida_anterior.corrida_aux.iloc[-1])):
#        corrida_anterior.dropna(inplace=True)
        
    #CORRIDA ACTUAL
    corrida_actual = query[(query['id_sc2_corrida']== row['id_sc2_corrida']+1)&(query['mes'] == row['mes'])]
    corrida_actual.set_index(['id_sc2_producto','talle','tipo_vigente','color'], inplace=True)
    corrida_actual.reset_index(inplace=True)
    
#    if (corrida_actual.corrida_aux.isna().sum() == len(corrida_actual)-1
#        and not np.isnan(corrida_actual.corrida_aux.iloc[-1])):       
#        corrida_actual.dropna(inplace=True)
        
        
    if len(corrida_actual) == 0 or len(corrida_anterior) == 0 :
        print(row, 'QUE PASO ACA?')
        
        continue
    
        
    #copio cualquiera de los dos df para agregar una columna 'resta_stock' y no tocar los df para operar.
    PROBANDO = corrida_actual.copy()              

    corrida_actual.fillna(0,inplace=True)
    corrida_anterior.fillna(0,inplace=True)
    
    #Resta_stock : δStock ↓
   #EDIT 29/10 

    PROBANDO["resta_stock"] = np.where(corrida_actual['stock'] < corrida_anterior['stock'],
                                       (np.where(corrida_actual['stock'] > 0 ,corrida_anterior['stock'] - corrida_actual['stock'],0) ),0)
    
    #resta_tmpstp : δDias ↓
    #EDIT 29/10

    PROBANDO["resta_tmstp"] = np.where(corrida_actual['stock'] < corrida_anterior['stock'],
                                       (np.where(corrida_actual['stock'] > 0,
                                       corridas_fechas.loc[index+1,'fecha_alta']-corridas_fechas.loc[index,'fecha_alta'],
                                       0) ),0)
    
    PROBANDO['SUMA_HIGH_STOCK'] = corrida_actual['high_stock'] + corrida_anterior['high_stock']
    
    PROBANDO['SUMA_FALTA_CORRIDA'] = corrida_actual['falta_corrida'] + corrida_anterior['falta_corrida']
    


    #tomar la fecha cuando fecha_alta es NaN  .()

    #regex para tomar δdias↓ int 
    PROBANDO['delta_dias_resta'] = PROBANDO['resta_tmstp'].astype(str).str.extract(r"([\d]+)", expand=False).astype(int)                                   

    #suma_stock : δStock ↑
    #EDIT 29/10
    
    PROBANDO["suma_stock"] = np.where(corrida_actual['stock'] > corrida_anterior['stock'],
                                      (np.where(corrida_anterior['stock'] > 0 ,
                                                corrida_actual['stock'] - corrida_anterior['stock'],0) ),0)
    
    #suma_tmstp : δDias ↑
    #EDIT 29/10
    PROBANDO["suma_tmstp"] = np.where(corrida_actual['stock'] > corrida_anterior['stock'],
                                      (np.where(corrida_anterior['stock'] > 0,
                                                corridas_fechas.loc[index+1,'fecha_alta']-corridas_fechas.loc[index,'fecha_alta'],
                                                0)),0)



    #regex para tomar δDias↑ int
    PROBANDO['delta_dias_suma'] = PROBANDO['suma_tmstp'].astype(str).str.extract(r"([\d]+)", expand=False).astype(int)                                   
    #valor para sumar stock siempre que suba
    
    #EDIT 29/10
    
    PROBANDO["contar_suba"] = np.where(corrida_actual['stock'] > corrida_anterior['stock'],
                                       (np.where(corrida_anterior['stock'] > 0,1,0)),0)
    
    
    PROBANDO['mes'] = row['mes']
    # solo en la primer vuelta
    if index == 0:
        resultado = PROBANDO.copy()
    if index > 0:
        resultado = resultado.append(PROBANDO,sort=True)



/home/aa/miniconda3/envs/dev_env/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


fecha_alta        2019-06-27 00:00:00
id_sc2_corrida                      3
timestamp             6 days 00:00:00
delta_dias                          6
mes                                 6
Name: 2, dtype: object QUE PASO ACA?
fecha_alta        2019-07-30 00:00:00
id_sc2_corrida                      7
timestamp             8 days 00:00:00
delta_dias                          8
mes                                 7
Name: 6, dtype: object QUE PASO ACA?
fecha_alta        2019-08-28 00:00:00
id_sc2_corrida                     13
timestamp             6 days 00:00:00
delta_dias                          6
mes                                 8
Name: 12, dtype: object QUE PASO ACA?
fecha_alta        2019-09-30 00:00:00
id_sc2_corrida                     19
timestamp             4 days 00:00:00
delta_dias                          4
mes                                 9
Name: 18, dtype: object QUE PASO ACA?
fecha_alta        2019-10-23 00:00:00
id_sc2_corrida                     22
timestamp     

In [34]:
resultado['mes'].unique()

array([ 6,  7,  8,  9, 10, 11])

In [35]:
query_copiado.head()
#resultado.append(PROBANDO,sort=True)

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta,corrida_aux,stock,precio,precio_original,TIPO,origen
5691402,1,Lingerie150904001,L,Black,TOP,2019-06-12,1.0,36.0,8.0,8.0,TOP,SHEIN
6897019,1,Lingerie150904001,M,Black,TOP,2019-06-12,1.0,50.0,8.0,8.0,TOP,SHEIN
1041920,1,Lingerie150904001,S,Black,TOP,2019-06-12,1.0,59.0,8.0,8.0,TOP,SHEIN
12407715,1,NCSTD14123152,L,Black,POLLERA,NaT,NaN,NaN,NaN,NaN,None,None
12755189,1,NCSTD14123152,M,Black,POLLERA,NaT,NaN,NaN,NaN,NaN,None,None


In [36]:
#Solo estan una vez, no podría sacar indicadores?
solos = query_copiado[query_copiado.set_index(['id_sc2_producto','talle','color','tipo_vigente']).index.isin(df_quitar.set_index(['id_sc2_producto','talle','color','tipo_vigente']).index)]

In [37]:
solos.dropna(inplace=True)
solos.reset_index(drop=True,inplace=True)

/home/aa/miniconda3/envs/dev_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
resultado.columns

Index(['SUMA_FALTA_CORRIDA', 'SUMA_HIGH_STOCK', 'TIPO', 'color', 'contar_suba',
       'corrida_aux', 'delta_dias', 'delta_dias_resta', 'delta_dias_suma',
       'falta_corrida', 'fecha_alta_x', 'fecha_alta_y', 'high_stock',
       'id_sc2_corrida', 'id_sc2_producto', 'mes', 'origen', 'precio',
       'precio_original', 'resta_stock', 'resta_tmstp', 'stock', 'suma_stock',
       'suma_tmstp', 'talle', 'timestamp', 'tipo_vigente'],
      dtype='object')

### ** FTEST

In [39]:
#LE AGREGUE FECHA_ALTA_Y
V=resultado[['SUMA_FALTA_CORRIDA','SUMA_HIGH_STOCK','id_sc2_producto','id_sc2_corrida','talle','tipo_vigente','mes','resta_stock',
             'suma_stock','stock','delta_dias','delta_dias_resta','delta_dias_suma','color',
             'contar_suba']].fillna(0)

In [40]:
resultado.columns

Index(['SUMA_FALTA_CORRIDA', 'SUMA_HIGH_STOCK', 'TIPO', 'color', 'contar_suba',
       'corrida_aux', 'delta_dias', 'delta_dias_resta', 'delta_dias_suma',
       'falta_corrida', 'fecha_alta_x', 'fecha_alta_y', 'high_stock',
       'id_sc2_corrida', 'id_sc2_producto', 'mes', 'origen', 'precio',
       'precio_original', 'resta_stock', 'resta_tmstp', 'stock', 'suma_stock',
       'suma_tmstp', 'talle', 'timestamp', 'tipo_vigente'],
      dtype='object')

In [ ]:
#esto no
#
#    FALTA CORRIDA Y FLAG STOCK, NO ESTAN CONTEMPLADOS, AUN
#    V=resultado[['id_sc2_producto','id_sc2_corrida','talle','tipo_vigente','mes','resta_stock',
#             'suma_stock','stock','delta_dias','delta_dias_resta','delta_dias_suma','color',
#             'high_stock','contar_suba','falta_corrida']].fillna(0)

In [31]:
V[(V['id_sc2_producto']=='Lingerie150904001')&(V['talle'] =='L')&(V['mes'] == 6)]

,id_sc2_producto,id_sc2_corrida,talle,tipo_vigente,mes,resta_stock,suma_stock,stock,delta_dias,delta_dias_resta,delta_dias_suma,color,high_stock,contar_suba,falta_corrida
0,Lingerie150904001,2,L,TOP,6,5.0,0.0,31.0,9,9,0,Black,0,0,0
0,Lingerie150904001,3,L,TOP,6,0.0,0.0,0.0,6,0,0,Black,0,0,1


In [32]:
query[(query['id_sc2_producto']=='Lingerie150904001')&(query['talle'] =='L')&(query['mes'] == 6)]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta_x,corrida_aux,stock,precio,precio_original,TIPO,origen,high_stock,falta_corrida,fecha_alta_y,timestamp,delta_dias,mes
0,1,Lingerie150904001,L,Black,TOP,2019-06-12,1.0,36.0,8.0,8.0,TOP,SHEIN,0,0,2019-06-12,0,0,6
539248,2,Lingerie150904001,L,Black,TOP,2019-06-21,2.0,31.0,8.0,8.0,TOP,SHEIN,0,0,2019-06-21,9 days 00:00:00,9,6
1078496,3,Lingerie150904001,L,Black,TOP,NaT,NaN,NaN,NaN,NaN,None,None,0,1,2019-06-27,6 days 00:00:00,6,6


In [41]:
start_ = datetime.datetime.today()
F=V.groupby(['id_sc2_producto','talle','tipo_vigente','color','mes']).sum()
end_ = datetime.datetime.today()
#[["demanda_orig", "devolucion_orig"]].apply(lambda x : x.sum()).reset_index()

In [36]:
len(F)

3235488

In [48]:
len(V)
#10245712

10784960

In [42]:
F = F.reset_index()

In [76]:
QUERY.head()

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta_x,corrida_aux,stock,precio,precio_original,TIPO,origen,fecha_alta_y,timestamp,delta_dias,mes
0,1,Lingerie150904001,L,Black,TOP,2019-06-12,1.0,36.0,8.0,8.0,TOP,SHEIN,2019-06-12,0,0,6
1,1,Lingerie150904001,M,Black,TOP,2019-06-12,1.0,50.0,8.0,8.0,TOP,SHEIN,2019-06-12,0,0,6
2,1,Lingerie150904001,S,Black,TOP,2019-06-12,1.0,59.0,8.0,8.0,TOP,SHEIN,2019-06-12,0,0,6
3,1,NCSTD14123152,L,Black,POLLERA,NaT,NaN,NaN,NaN,NaN,None,None,2019-06-12,0,0,6
4,1,NCSTD14123152,M,Black,POLLERA,NaT,NaN,NaN,NaN,NaN,None,None,2019-06-12,0,0,6


In [51]:
QUERY[
    (QUERY['id_sc2_producto'] == 'RBLO170605005')&
    (QUERY['talle'] == 'one-size')&
    (QUERY['color'] == 'White')&
    (QUERY['tipo_vigente'] == 'BLUSA')&
    (QUERY['mes'] == 7)
    
]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta_x,corrida_aux,stock,precio,precio_original,TIPO,origen,fecha_alta_y,timestamp,delta_dias,mes
1617818,4,RBLO170605005,one-size,White,BLUSA,2019-07-01,4.0,999.0,13.0,13.0,BLUSA,SHEIN,2019-07-01,4 days 00:00:00,4,7
2157066,5,RBLO170605005,one-size,White,BLUSA,2019-07-10,5.0,55.0,13.0,13.0,BLUSA,SHEIN,2019-07-10,9 days 00:00:00,9,7
2696314,6,RBLO170605005,one-size,White,BLUSA,2019-07-22,6.0,84.0,13.0,13.0,BLUSA,SHEIN,2019-07-22,12 days 00:00:00,12,7
3235562,7,RBLO170605005,one-size,White,BLUSA,2019-07-30,7.0,78.0,13.0,13.0,BLUSA,SHEIN,2019-07-30,8 days 00:00:00,8,7


In [50]:
F[(F['SUMA_HIGH_STOCK'] == 1)&
  (F['SUMA_FALTA_CORRIDA'] == 0)
 ]

,id_sc2_producto,talle,tipo_vigente,color,mes,SUMA_FALTA_CORRIDA,SUMA_HIGH_STOCK,id_sc2_corrida,resta_stock,suma_stock,stock,delta_dias,delta_dias_resta,delta_dias_suma,contar_suba
445,RBLO170605005,one-size,BLUSA,White,7,0,1,18,950.0,29.0,217.0,29,17,12,1
981,RBLO170825002,L,BLUSA,Black and White,9,0,1,85,956.0,30.0,361.0,27,21,6,1
987,RBLO170825002,M,BLUSA,Black and White,9,0,1,85,937.0,59.0,511.0,27,23,4,1
993,RBLO170825002,S,BLUSA,Black and White,9,0,1,85,952.0,41.0,428.0,27,17,10,2
1224,RDRE170406103,L,VESTIDO,Burgundy,6,0,1,5,997.0,0.0,6.0,15,15,0,0
1230,RDRE170406103,M,VESTIDO,Burgundy,6,0,1,5,996.0,0.0,13.0,15,15,0,0
1236,RDRE170406103,S,VESTIDO,Burgundy,6,0,1,5,998.0,0.0,2.0,15,9,0,0
1599,RDRE170712102,L,VESTIDO,White,9,0,1,85,966.0,0.0,222.0,27,27,0,0
1611,RDRE170712102,M,VESTIDO,White,9,0,1,85,964.0,0.0,249.0,27,27,0,0
1623,RDRE170712102,S,VESTIDO,White,9,0,1,85,950.0,0.0,348.0,27,27,0,0


In [43]:
calculo_delta_dias = V.groupby(['id_sc2_producto','talle','tipo_vigente','color','mes'])

In [44]:
Lista_deltas = []
for i in calculo_delta_dias:
    
    #Aux para sacar el id_sc2_corrida A SALTEAR
    aux_reverse = i[1].reset_index(drop=True)[::-1].reset_index(drop=True)
    if aux_reverse.iloc[0]['stock'] == 0:
        CORRIDA_A_SALTEAR = aux_reverse.loc[0,'id_sc2_corrida']
        #DELTA DIAS SUM : LA SUMA DE TODOS LOS DELTA_DIAS "SACANDO DE LADO SI EL ULTIMO ES NaN"
        DELTA_DIAS_SUM = i[1][i[1]['id_sc2_corrida'] != CORRIDA_A_SALTEAR]['delta_dias'].sum()
    else:
        DELTA_DIAS_SUM = i[1]['delta_dias'].sum()
        
    #"id_sc2_producto","talle","tipo","color","mes","DELTA_DIAS_SUM"
    Lista_deltas.append([i[0][0],i[0][1],i[0][2],i[0][3],i[0][4],DELTA_DIAS_SUM])
        

In [59]:
Lista_deltas[1]

['Lingerie150904001', 'L', 'TOP', 'Black', 7, 29]

In [45]:
deltas = pd.DataFrame(Lista_deltas)

In [46]:
deltas.rename(columns={0:'id_sc2_producto',
                      1:'talle',
                      2:'TIPO_VIGENTE',
                      3:'color',
                      4:'mes',
                      5:'DELTA_DIAS_SUM'},inplace=True)

In [47]:
F = pd.merge(F,deltas,
                 left_on=['id_sc2_producto','talle','tipo_vigente','color','mes'],
                 right_on=['id_sc2_producto','talle','TIPO_VIGENTE','color','mes'],
                 how='inner')

In [48]:
F.columns

Index(['id_sc2_producto', 'talle', 'tipo_vigente', 'color', 'mes',
       'SUMA_FALTA_CORRIDA', 'SUMA_HIGH_STOCK', 'id_sc2_corrida',
       'resta_stock', 'suma_stock', 'stock', 'delta_dias', 'delta_dias_resta',
       'delta_dias_suma', 'contar_suba', 'TIPO_VIGENTE', 'DELTA_DIAS_SUM'],
      dtype='object')

In [49]:
F['demanda'] = F['resta_stock'] / F['delta_dias_resta']
F['abastecimiento'] = (F['suma_stock'] + F['delta_dias_suma'] * F['demanda'])/F['DELTA_DIAS_SUM']
F['abastecimiento'] = F['abastecimiento'].fillna(0)
F['demanda'] = F['demanda'].fillna(0)
F['origen'] = 'SHEIN'

In [44]:
if 1==3:
    F['demanda'] = F['resta_stock'] / F['delta_dias_resta']
    F['abastecimiento'] = (F['suma_stock'] + F['delta_dias_suma'] * F['demanda'])/F['delta_dias']
    F['abastecimiento'] = F['abastecimiento'].fillna(0)
    F['demanda'] = F['demanda'].fillna(0)
    F['origen'] = 'SHEIN'

In [45]:
#/V.groupby(['id_sc2_producto','id_sc2_corrida','talle','tipo_vigente','mes'])['delta_dias_resta']

In [46]:
if 1==3:
    resultado['demanda'] = resultado['resta_stock'] / resultado['delta_dias_resta']

    resultado['abastecimiento'] = (resultado['suma_stock'] + resultado['delta_dias_suma'] * resultado['demanda'])/delta_dias
    resultado['abastecimiento'] = resultado['abastecimiento'].fillna(0)
    resultado['demanda'] = resultado['demanda'].fillna(0)
    resultado['origen'] = 'SHEIN'

In [47]:
len(query_copiado)

14353430

In [48]:
len(F)

3235488

In [ ]:
####################### DISTINTOS
# distinct = 'select distinct top 1000  id_sc2_producto, talle, tipo_vigente, color from sc2_detalle '
# distinct = pd.read_sql(distinct,cnxn)
# DISTINCT = F[F.set_index(['id_sc2_producto','talle','color','tipo_vigente']).index.isin(distinct.set_index(['id_sc2_producto','talle','color','tipo_vigente']).index)]

In [59]:
query_copiado[
    (query_copiado['id_sc2_producto'] == 'RBLO170421101')&
    (query_copiado['talle'] == 'L')&
    (query_copiado['color'] == 'Multicolor')&
    (query_copiado['tipo_vigente'] == 'BLUSA')
]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta,corrida_aux,stock,precio,precio_original,TIPO,origen
7795466,1,RBLO170421101,L,Multicolor,BLUSA,2019-06-12,1.0,999.0,14.0,14.0,BLUSA,SHEIN
7795547,2,RBLO170421101,L,Multicolor,BLUSA,2019-06-21,2.0,999.0,14.0,28.0,BLUSA,SHEIN
7795370,3,RBLO170421101,L,Multicolor,BLUSA,2019-06-27,3.0,999.0,13.0,29.0,BLUSA,SHEIN
7801441,4,RBLO170421101,L,Multicolor,BLUSA,2019-07-01,4.0,999.0,14.0,14.0,BLUSA,SHEIN
7796402,5,RBLO170421101,L,Multicolor,BLUSA,2019-07-10,5.0,999.0,14.0,14.0,BLUSA,SHEIN
7794284,6,RBLO170421101,L,Multicolor,BLUSA,2019-07-22,6.0,999.0,14.0,14.0,BLUSA,SHEIN
7797270,7,RBLO170421101,L,Multicolor,BLUSA,NaT,NaN,NaN,NaN,NaN,None,None
7796771,8,RBLO170421101,L,Multicolor,BLUSA,2019-08-05,8.0,999.0,14.0,28.0,BLUSA,SHEIN
7794699,9,RBLO170421101,L,Multicolor,BLUSA,2019-08-08,9.0,999.0,14.0,28.0,BLUSA,SHEIN
7802166,10,RBLO170421101,L,Multicolor,BLUSA,2019-08-12,10.0,999.0,14.0,14.0,BLUSA,SHEIN


In [58]:
DISTINCT

,id_sc2_producto,talle,tipo_vigente,color,mes,id_sc2_corrida,resta_stock,suma_stock,stock,delta_dias,delta_dias_resta,delta_dias_suma,contar_suba,TIPO_VIGENTE,DELTA_DIAS_SUM,demanda,abastecimiento,origen,high_stock
312,RBLO170421101,L,BLUSA,Multicolor,6,5,0.0,0.0,1998.0,15,0,0,0,BLUSA,15,0.000000,0.000000,SHEIN,0
313,RBLO170421101,L,BLUSA,Multicolor,7,18,0.0,0.0,1998.0,29,0,0,0,BLUSA,21,0.000000,0.000000,SHEIN,0
314,RBLO170421101,L,BLUSA,Multicolor,8,55,0.0,0.0,4995.0,23,0,0,0,BLUSA,23,0.000000,0.000000,SHEIN,0
315,RBLO170421101,L,BLUSA,Multicolor,9,85,995.0,0.0,4000.0,27,4,0,0,BLUSA,27,248.750000,0.000000,SHEIN,0
316,RBLO170421101,L,BLUSA,Multicolor,10,43,8.0,0.0,4.0,13,13,0,0,BLUSA,13,0.615385,0.000000,SHEIN,0
317,RBLO170421101,L,BLUSA,Multicolor,11,75,0.0,0.0,0.0,14,0,0,0,BLUSA,10,0.000000,0.000000,SHEIN,0
318,RBLO170421101,M,BLUSA,Multicolor,6,5,0.0,0.0,1998.0,15,0,0,0,BLUSA,15,0.000000,0.000000,SHEIN,0
319,RBLO170421101,M,BLUSA,Multicolor,7,18,0.0,0.0,1998.0,29,0,0,0,BLUSA,21,0.000000,0.000000,SHEIN,0
320,RBLO170421101,M,BLUSA,Multicolor,8,55,0.0,0.0,4995.0,23,0,0,0,BLUSA,23,0.000000,0.000000,SHEIN,0
321,RBLO170421101,M,BLUSA,Multicolor,9,85,987.0,0.0,4008.0,27,4,0,0,BLUSA,27,246.750000,0.000000,SHEIN,0


In [12]:
# aux = query.loc[~query.tipo.isna(), ("id_sc2_producto", "tipo","talle","color")].set_index(["id_sc2_producto","talle","color"]).tipo
# resultado.set_index(["id_sc2_producto","talle","color"], inplace=True)
# aux = aux[~aux.index.duplicated()]
# resultado.tipo.fillna(aux, inplace=True)
#resultado.reset_index(inplace=True)

In [47]:
F.columns

Index(['index', 'id_sc2_producto', 'talle', 'tipo_vigente', 'color', 'mes',
       'id_sc2_corrida', 'resta_stock', 'suma_stock', 'stock', 'delta_dias',
       'delta_dias_resta', 'delta_dias_suma', 'high_stock', 'contar_suba',
       'falta_corrida', 'TIPO_VIGENTE', 'DELTA_DIAS_SUM', 'demanda',
       'abastecimiento', 'origen'],
      dtype='object')

In [50]:
F.head()

,id_sc2_producto,talle,tipo_vigente,color,mes,SUMA_FALTA_CORRIDA,SUMA_HIGH_STOCK,id_sc2_corrida,resta_stock,suma_stock,stock,delta_dias,delta_dias_resta,delta_dias_suma,contar_suba,TIPO_VIGENTE,DELTA_DIAS_SUM,demanda,abastecimiento,origen
0,Lingerie150904001,L,TOP,Black,6,1,0,5,5.0,0.0,31.0,15,9,0,0,TOP,9,0.555556,0.000000,SHEIN
1,Lingerie150904001,L,TOP,Black,7,0,0,18,18.0,15.0,61.0,29,17,12,1,TOP,29,1.058824,0.955375,SHEIN
2,Lingerie150904001,L,TOP,Black,8,0,0,55,16.0,0.0,50.0,23,19,0,0,TOP,23,0.842105,0.000000,SHEIN
3,Lingerie150904001,L,TOP,Black,9,0,0,85,0.0,0.0,5.0,27,0,0,0,TOP,27,0.000000,0.000000,SHEIN
4,Lingerie150904001,L,TOP,Black,10,0,0,43,0.0,0.0,2.0,13,0,0,0,TOP,13,0.000000,0.000000,SHEIN


In [51]:
F.drop('TIPO_VIGENTE',axis=1,inplace=True)

In [52]:
F.loc[F['SUMA_HIGH_STOCK'] > 0,'SUMA_HIGH_STOCK'] = 1
F.loc[F['SUMA_FALTA_CORRIDA'] > 0,'SUMA_FALTA_CORRIDA'] = 1


In [51]:
if 1==3:
    #CREO Q NO HACE FALTA ESTO XQ YA TENGO HIGH STOCK
    #agregué tipo_vigente al gpby
    #res_copy = resultado.copy()
    res_copy = F.copy()

    #AGRUPAR Y SUMAR LOS HIGH_STOCK
    HIGH_STOCK = query.groupby(['id_sc2_producto','talle','color','tipo_vigente'])['high_stock'].sum().reset_index()

    res_copy.drop('high_stock',axis=1,inplace=True)

    #MERGE PARA TENER HIGH_STOCK EN RES_COPY, QUE AHORA SE LLAMA MERGE_HIGH_STOCK
    MERGE_HIGH_STOCK = pd.merge(res_copy,HIGH_STOCK,on=['id_sc2_producto','talle','color','tipo_vigente'])
    #AGREGAR HIGH_STOCK Y PONER 1 SI ES MAYOR A 0
    MERGE_HIGH_STOCK.loc[MERGE_HIGH_STOCK['high_stock']>0,'high_stock'] = 1

    #AGRUPAR Y SUMAR LOS FALTA_CORRIDA
    FALTA_CORRIDA = query.groupby(['id_sc2_producto','talle','color','tipo_vigente'])['falta_corrida'].sum().reset_index()

In [52]:
if 1==3:
    #CREO QUE NO HACE FALTA, PORQUE YA TENGO ESTE CAMPO
    
    #MERGE_HIGH_STOCK.drop('falta_corrida', axis=1, inplace=True)

    #MERGE PARA TENER FALTA_CORRIDA EN EL NUEVO DF
    MERGE_FALTA_CORRIDA = pd.merge(MERGE_HIGH_STOCK,FALTA_CORRIDA,on=['id_sc2_producto','talle','color','tipo_vigente'])

    #AGREGAR FALTA_CORRIDA Y PONER 1 SI ES MAYOR A 0
    MERGE_FALTA_CORRIDA.loc[MERGE_FALTA_CORRIDA['falta_corrida'] > 0,'falta_corrida'] = 1

In [53]:
#ANTES ERA MERGE_FALTA_CORRIDA, EN VEZ DE F
F['fecha'] = fecha
F['desde_corrida'] = DESDE_CORRIDA
F['hasta_corrida'] = HASTA_CORRIDA

In [54]:
F.rename(columns={'SUMA_FALTA_CORRIDA':'falta_corrida',
                 'SUMA_HIGH_STOCK':'high_stock'},inplace=True)
F.columns

Index(['id_sc2_producto', 'talle', 'tipo_vigente', 'color', 'mes',
       'falta_corrida', 'high_stock', 'id_sc2_corrida', 'resta_stock',
       'suma_stock', 'stock', 'delta_dias', 'delta_dias_resta',
       'delta_dias_suma', 'contar_suba', 'DELTA_DIAS_SUM', 'demanda',
       'abastecimiento', 'origen', 'fecha', 'desde_corrida', 'hasta_corrida'],
      dtype='object')

In [55]:
#MERGE_FALTA_CORRIDA[['origen','id_sc2_producto','talle','tipo_vigente','color','mes','demanda','abastecimiento','contar_suba','desde_corrida','hasta_corrida','fecha','falta_corrida','high_stock']]

In [56]:
from sqlalchemy import create_engine
import urllib
params = urllib.parse.quote_plus('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+
                                 server+';DATABASE='+database+';UID='+username+';PWD='+ password)

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [57]:
TOSQL = F[['origen','id_sc2_producto','talle','tipo_vigente','color','demanda','abastecimiento','contar_suba','desde_corrida','hasta_corrida','fecha','falta_corrida','high_stock','mes']]

TOSQL=TOSQL.rename(columns={'high_stock':'flag_stock'})
TOSQL=TOSQL.rename(columns={'falta_corrida':'flag_corrida'})
#TOSQL=TOSQL.rename(columns={'fecha_alta':'fecha_scraper'})
TOSQL=TOSQL.rename(columns={'fecha':'fecha_alta'})
TOSQL=TOSQL.rename(columns={'contar_suba':'cantidad_abastecimiento'})

In [58]:
TOSQL['fecha_alta'].head()

0    2019-11-28
1    2019-11-28
2    2019-11-28
3    2019-11-28
4    2019-11-28
Name: fecha_alta, dtype: object

In [59]:
TOSQL.reset_index(drop=True,inplace=True)

In [60]:
GROUPBY_CALCULO_MM = query.groupby(['id_sc2_producto','talle','color','tipo_vigente'])

In [61]:
lista_calculo = []
for grupo in GROUPBY_CALCULO_MM:
    
    aux_grupo =  grupo[1].dropna(0)
    #id_sc2_producto, talle, color, tipo_vigente
    lista_calculo.append([
                        grupo[0][0],
                        grupo[0][1],
                        grupo[0][2],
                        grupo[0][3],
                        aux_grupo['fecha_alta_y'].min(),
                        aux_grupo['fecha_alta_y'].max()
                        ])

In [62]:
dfMM = pd.DataFrame(lista_calculo)
dfMM.rename(columns = {0:'id_sc2_producto',
                      1:'talle',
                      2:'color',
                      3:'tipo_vigente',
                      4:'fecha_min',
                      5:'fecha_max'},inplace=True)

In [63]:
merge_MM = pd.merge(TOSQL,dfMM,on=['id_sc2_producto','talle','color','tipo_vigente'],how='inner')

In [64]:
len(merge_MM) 

3235488

In [145]:
#TOSQL.rename(columns={'fecha_alta_x':'fecha_alta'},inplace=True)
#TOSQL.rename(columns={'fecha_alta_y':'primer_aparicion'},inplace=True)

In [65]:
merge_MM.rename(columns={'fecha_min':'primer_aparicion'},inplace=True)
merge_MM.rename(columns={'fecha_max':'ultima_aparicion'},inplace=True)

In [66]:
merge_MM.columns

Index(['origen', 'id_sc2_producto', 'talle', 'tipo_vigente', 'color',
       'demanda', 'abastecimiento', 'cantidad_abastecimiento', 'desde_corrida',
       'hasta_corrida', 'fecha_alta', 'flag_corrida', 'flag_stock', 'mes',
       'primer_aparicion', 'ultima_aparicion'],
      dtype='object')

In [67]:
TOSQL = merge_MM[['origen','id_sc2_producto','talle','tipo_vigente','color','mes','demanda','abastecimiento','cantidad_abastecimiento',
              'desde_corrida','hasta_corrida','fecha_alta','flag_corrida','flag_stock','primer_aparicion','ultima_aparicion']]

In [68]:
COPY = TOSQL.copy()

In [69]:
TOTAL_REGISTROS = len(TOSQL)

In [70]:
CANTIDAD_BLOQUES = 4

BLOQUE = TOTAL_REGISTROS / CANTIDAD_BLOQUES

BLOQUE

808872.0

In [71]:
BLOQUE = int(BLOQUE)

In [72]:
BLOQUE

808872

In [73]:
#split desde chunksize 
split_df = lambda items, chunksize: [items[i:i+chunksize] for i in range(0, len(items), chunksize)]

In [74]:

L = split_df(COPY,BLOQUE)

In [75]:
len(L[0])+len(L[1])+len(L[2])+len(L[3]) == len(COPY)

True

In [80]:
L[0].to_sql("sc_indicadores",engine, index=False,if_exists='replace') #append
#len(TOSQL)

In [81]:
L[1].to_sql("sc_indicadores",engine, index=False,if_exists='append') #append
#len(TOSQL)

In [82]:
L[2].to_sql("sc_indicadores",engine, index=False,if_exists='append') #append

In [83]:
L[3].to_sql("sc_indicadores",engine, index=False,if_exists='append') #append

In [86]:
TOSQL[(TOSQL['id_sc2_producto']=='swblouse00190624947')&
             (TOSQL['talle'] == 'M')&
             (TOSQL['color'] == 'White')]

,origen,id_sc2_producto,talle,tipo_vigente,color,mes,demanda,abastecimiento,cantidad_abastecimiento,desde_corrida,hasta_corrida,fecha_alta,flag_corrida,flag_stock,primer_aparicion,ultima_aparicion
936546,SHEIN,swblouse00190624947,M,BLUSA,White,6,0.0,0.000000,0,1,26,2019-11-28,1,0,2019-07-10,2019-09-30
936547,SHEIN,swblouse00190624947,M,BLUSA,White,7,0.0,0.000000,0,1,26,2019-11-28,1,1,2019-07-10,2019-09-30
936548,SHEIN,swblouse00190624947,M,BLUSA,White,8,0.0,0.000000,0,1,26,2019-11-28,0,1,2019-07-10,2019-09-30
936549,SHEIN,swblouse00190624947,M,BLUSA,White,9,499.0,406.777778,3,1,26,2019-11-28,0,1,2019-07-10,2019-09-30
936550,SHEIN,swblouse00190624947,M,BLUSA,White,10,0.0,0.000000,0,1,26,2019-11-28,1,0,2019-07-10,2019-09-30
936551,SHEIN,swblouse00190624947,M,BLUSA,White,11,0.0,0.000000,0,1,26,2019-11-28,1,0,2019-07-10,2019-09-30


In [88]:
TOSQL[(TOSQL['id_sc2_producto']=='blazer14071803')&
             (TOSQL['talle'] == 'L')&
             (TOSQL['color'] == 'Black and White')]

,origen,id_sc2_producto,talle,tipo_vigente,color,mes,demanda,abastecimiento,cantidad_abastecimiento,desde_corrida,hasta_corrida,fecha_alta,flag_corrida,flag_stock,primer_aparicion,ultima_aparicion
9822,SHEIN,blazer14071803,L,BLAZER,Black and White,6,0.000000,0.000000,0,1,26,2019-11-28,1,1,2019-06-21,2019-11-15
9823,SHEIN,blazer14071803,L,BLAZER,Black and White,7,0.000000,0.000000,0,1,26,2019-11-28,0,1,2019-06-21,2019-11-15
9824,SHEIN,blazer14071803,L,BLAZER,Black and White,8,1.500000,0.000000,0,1,26,2019-11-28,1,0,2019-06-21,2019-11-15
9825,SHEIN,blazer14071803,L,BLAZER,Black and White,9,3.666667,5.555556,2,1,26,2019-11-28,0,0,2019-06-21,2019-11-15
9826,SHEIN,blazer14071803,L,BLAZER,Black and White,10,4.333333,2.871795,1,1,26,2019-11-28,0,0,2019-06-21,2019-11-15
9827,SHEIN,blazer14071803,L,BLAZER,Black and White,11,4.555556,1.984127,1,1,26,2019-11-28,0,0,2019-06-21,2019-11-15


In [ ]:
blazer14071803
L
Black and White

In [90]:
QUERY[(QUERY['id_sc2_producto']=='blazer14071803')&
             (QUERY['talle'] == 'L')&
             (QUERY['color'] == 'Black and White')]


,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta_x,corrida_aux,stock,precio,precio_original,TIPO,origen,fecha_alta_y,timestamp,delta_dias,mes
1637,1,blazer14071803,L,Black and White,BLAZER,NaT,NaN,NaN,NaN,NaN,None,None,2019-06-12,0,0,6
540885,2,blazer14071803,L,Black and White,BLAZER,2019-06-21,2.0,999.0,26.0,26.0,BLAZER,SHEIN,2019-06-21,9 days 00:00:00,9,6
1080133,3,blazer14071803,L,Black and White,BLAZER,2019-06-27,3.0,999.0,26.0,26.0,BLAZER,SHEIN,2019-06-27,6 days 00:00:00,6,6
1619381,4,blazer14071803,L,Black and White,BLAZER,2019-07-01,4.0,999.0,26.0,26.0,BLAZER,SHEIN,2019-07-01,4 days 00:00:00,4,7
2158629,5,blazer14071803,L,Black and White,BLAZER,2019-07-10,5.0,999.0,24.0,24.0,BLAZER,SHEIN,2019-07-10,9 days 00:00:00,9,7
2697877,6,blazer14071803,L,Black and White,BLAZER,2019-07-22,6.0,999.0,24.0,24.0,BLAZER,SHEIN,2019-07-22,12 days 00:00:00,12,7
3237125,7,blazer14071803,L,Black and White,BLAZER,2019-07-30,7.0,999.0,24.0,24.0,BLAZER,SHEIN,2019-07-30,8 days 00:00:00,8,7
3776373,8,blazer14071803,L,Black and White,BLAZER,2019-08-05,8.0,1.0,24.0,24.0,BLAZER,SHEIN,2019-08-05,6 days 00:00:00,6,8
4315621,9,blazer14071803,L,Black and White,BLAZER,2019-08-08,9.0,1.0,24.0,24.0,BLAZER,SHEIN,2019-08-08,3 days 00:00:00,3,8
4854869,10,blazer14071803,L,Black and White,BLAZER,2019-08-12,10.0,1.0,24.0,24.0,BLAZER,SHEIN,2019-08-12,4 days 00:00:00,4,8


In [165]:
TOSQL[(TOSQL['id_sc2_producto']=='Lingerie150904001')&(TOSQL['talle']=='L')&(TOSQL['tipo_vigente']=='CAMISETAS S/MANGA')
      &(TOSQL['color']=='Black')]

,origen,id_sc2_producto,talle,tipo_vigente,color,demanda,abastecimiento,cantidad_abastecimiento,desde_corrida,hasta_corrida,fecha_alta,flag_corrida,flag_stock,mes


In [366]:
query[(query['id_sc2_producto']=='Lingerie150904001')&(query['talle']=='L')&(query['tipo_vigente']=='CAMISETAS S/MANGA')
      &(query['color']=='Black')&(query['mes']==6)]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo_vigente,fecha_alta_x,corrida_aux,stock,precio,precio_original,TIPO,origen,high_stock,falta_corrida,fecha_alta_y,timestamp,delta_dias,mes
0,1,Lingerie150904001,L,Black,CAMISETAS S/MANGA,2019-06-12,1.0,36.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0,2019-06-12,0,0,6
474119,2,Lingerie150904001,L,Black,CAMISETAS S/MANGA,2019-06-21,2.0,31.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0,2019-06-21,9 days 00:00:00,9,6
948238,3,Lingerie150904001,L,Black,CAMISETAS S/MANGA,NaT,NaN,NaN,NaN,NaN,None,None,0,1,2019-06-27,6 days 00:00:00,6,6


In [378]:
36/(27-12)

2.4

In [376]:
0.5555555+5.166667

5.7222225

In [255]:
TOSQL[(TOSQL['flag_corrida']==0 )& (TOSQL['flag_stock']==0)]

,origen,id_sc2_producto,talle,color,demanda,abastecimiento,cantidad_abastecimiento,desde_corrida,hasta_corrida,fecha_alta,flag_corrida,flag_stock
7,SHEIN,RBLA170907201,L,Red,3.437500,3.994201,9,1,16,2019-09-19,0,0
8,SHEIN,RBLA170907201,M,Red,6.333333,7.147766,7,1,16,2019-09-19,0,0
9,SHEIN,RBLA170907201,S,Red,4.054545,4.714339,8,1,16,2019-09-19,0,0
10,SHEIN,RBLO160406103,L,Army Green,1.481013,2.171734,3,1,16,2019-09-19,0,0
11,SHEIN,RBLO160406103,M,Army Green,2.166667,1.836770,3,1,16,2019-09-19,0,0
12,SHEIN,RBLO160406103,S,Army Green,2.012195,1.785140,2,1,16,2019-09-19,0,0
13,SHEIN,RBLO160406103,XL,Army Green,1.413793,1.991113,5,1,16,2019-09-19,0,0
15,SHEIN,RBLO160608102,L,Bright,1.513158,0.393380,1,1,16,2019-09-19,0,0
16,SHEIN,RBLO160608102,L,Green,1.513158,0.393380,1,1,16,2019-09-19,0,0
17,SHEIN,RBLO160608102,M,Bright,2.227848,1.104137,3,1,16,2019-09-19,0,0


In [ ]:
******************************************************************

In [270]:
PRUEBO_UNO = query[(query['id_sc2_producto']=='RBLO170421101')&(query['talle']=='L')]

In [284]:
corridas_fechas

,fecha_alta,id_sc2_corrida,timestamp,delta_dias
0,2019-06-12,1,0,0
1,2019-06-21,2,9 days 00:00:00,9
2,2019-06-27,3,6 days 00:00:00,6
3,2019-07-01,4,4 days 00:00:00,4
4,2019-07-10,5,9 days 00:00:00,9
5,2019-07-22,6,12 days 00:00:00,12
6,2019-07-30,7,8 days 00:00:00,8
7,2019-08-05,8,6 days 00:00:00,6
8,2019-08-08,9,3 days 00:00:00,3
9,2019-08-12,10,4 days 00:00:00,4


In [306]:
#PASO CORRIDA SOLA (ES MAS LARGO Y POCO ENTENDIBLE)
#corridas_fechas[corridas_fechas['id_sc2_corrida']==2].reset_index()['fecha_alta'][0]

#PASO CORRIDA -1 (ES MAS CORTO)
corridas_fechas.loc[13-1,'fecha_alta']



Timestamp('2019-08-28 00:00:00')

In [310]:
query[(query['id_sc2_producto']=='RBLO170704136')&(query['talle'] =='S')&(query['color']=='Bright')&(query['tipo']=='TOP')]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo,fecha_alta,corrida_aux,stock,precio,precio_original,TIPO,origen,high_stock,falta_corrida
2864385,1,RBLO170704136,S,Bright,TOP,2019-06-12,1.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2862912,2,RBLO170704136,S,Bright,TOP,NaT,NaN,NaN,NaN,NaN,None,None,0,1
2863157,3,RBLO170704136,S,Bright,TOP,2019-06-27,3.0,999.0,10.0,20.0,TOP,SHEIN,1,0
2864531,4,RBLO170704136,S,Bright,TOP,2019-07-01,4.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2869924,5,RBLO170704136,S,Bright,TOP,2019-07-10,5.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2865857,6,RBLO170704136,S,Bright,TOP,2019-07-22,6.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2752808,7,RBLO170704136,S,Bright,TOP,2019-07-30,7.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2866066,8,RBLO170704136,S,Bright,TOP,2019-08-05,8.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2863731,9,RBLO170704136,S,Bright,TOP,2019-08-08,9.0,999.0,10.0,10.0,TOP,SHEIN,1,0
2866735,10,RBLO170704136,S,Bright,TOP,NaT,NaN,NaN,NaN,NaN,None,None,0,1


In [246]:
len(corridas_fechas)

16

In [278]:
query[query['tipo'].isnull()]

,id_sc2_corrida,id_sc2_producto,talle,color,tipo,fecha_alta,corrida_aux,stock,precio,precio_original,TIPO,origen,high_stock,falta_corrida


In [205]:
query['stock'] = query['stock'].fillna(0)

In [233]:
query['color'].unique()

array(['Black', 'Red', 'Army Green', 'Multicolor', 'Bright', 'Green',
       'White', 'Black and White', 'Yellow', 'Pink', 'Blue', 'Ginger',
       'Navy', 'Burgundy', 'Maroon', 'Grey', 'Pastel', 'Apricot', 'Beige',
       'Mustard', 'Turquoise', 'Purple', 'Brown', 'Coffee', 'Rust',
       'Khaki', 'Orange', 'Hot Pink', 'Gold', 'Silver', 'Nude', 'Camel',
       'Neon', 'Champagne', 'Bronze', 'Ombre', 'Único', 'Peach'],
      dtype=object)

In [159]:
TEMP = query[query['stock'] > 0]

In [160]:
FECHA_INICIO = pd.DataFrame(TEMP.groupby(['id_sc2_producto','talle','color','tipo'])['fecha_alta'].min().reset_index())
FECHA_MAX = pd.DataFrame(TEMP.groupby(['id_sc2_producto','talle','color','tipo'])['fecha_alta'].max().reset_index())

In [164]:
AA = pd.merge(FECHA_MAX,FECHA_INICIO,on=['id_sc2_producto','talle','color','tipo'],how='inner')

In [165]:
len(AA)

412966

In [157]:
#FECHA_INICIO.drop_duplicates(['id_sc2_producto','talle','color','tipo'])

412966

In [142]:
#query['PRIMERA'] =[ 1 if row['stock'] > 0 else 0 for index,row in query.iterrows() ]

In [124]:
FALTANTES = query[query['falta_corrida'] == 1]

In [126]:
#FALTANTES
TEMP1 = pd.merge(FALTANTES,corridas_fechas,on='id_sc2_corrida',how='left').drop('fecha_alta_x',axis=1).rename(columns={'fecha_alta_y':'fecha_alta'})

In [123]:
#gby = 
query.groupby(['id_sc2_producto','talle','color','tipo'],as_index=False)

datetime.timedelta(3)

In [31]:
query[(query['id_sc2_producto']=='Lingerie150904001')&(query['talle']=='L')&(query['color']=='Black')].sort_values('id_sc2_corrida')

,id_sc2_corrida,id_sc2_producto,talle,color,fecha_alta,corrida_aux,stock,precio,precio_original,tipo,origen,high_stock,falta_corrida
4056430,1,Lingerie150904001,L,Black,2019-06-12,1.0,36.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
3555022,2,Lingerie150904001,L,Black,2019-06-21,2.0,31.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
3449415,3,Lingerie150904001,L,Black,NaT,NaN,NaN,NaN,NaN,None,None,0,1
2564195,4,Lingerie150904001,L,Black,2019-07-01,4.0,23.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
3986753,5,Lingerie150904001,L,Black,2019-07-10,5.0,13.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
3861823,6,Lingerie150904001,L,Black,2019-07-22,6.0,28.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
2719069,7,Lingerie150904001,L,Black,NaT,NaN,NaN,NaN,NaN,None,None,0,1
3735992,8,Lingerie150904001,L,Black,2019-08-05,8.0,19.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
3532869,9,Lingerie150904001,L,Black,2019-08-08,9.0,15.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
4832214,10,Lingerie150904001,L,Black,2019-08-12,10.0,15.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0


In [34]:
query.groupby(['id_sc2_producto','talle','color','tipo'])['fecha_alta'].min()

id_sc2_producto    talle     color            tipo             
Lingerie150904001  L         Black            CAMISETAS S/MANGA   2019-06-12
                             Sexy             CAMISETAS S/MANGA   2019-07-30
                   M         Black            CAMISETAS S/MANGA   2019-06-12
                             Sexy             CAMISETAS S/MANGA   2019-07-30
                   S         Black            CAMISETAS S/MANGA   2019-06-12
                             Sexy             CAMISETAS S/MANGA   2019-07-30
RBLA170907201      L         Red              BLAZER              2019-06-12
                   M         Red              BLAZER              2019-06-12
                   S         Red              BLAZER              2019-06-12
RBLO160406103      L         Army Green       BLUSA               2019-06-12
                   M         Army Green       BLUSA               2019-06-12
                   S         Army Green       BLUSA               2019-06-12
            

In [46]:
corrida_actual[corrida_actual['stock'].isnull()]

,id_sc2_producto,talle,tipo,color,id_sc2_corrida,fecha_alta,corrida_aux,stock,precio,precio_original,origen,high_stock,falta_corrida


In [76]:
query[query['stock']<1]

,id_sc2_corrida,id_sc2_producto,talle,color,fecha_alta,corrida_aux,stock,precio,precio_original,tipo,origen,high_stock,falta_corrida


In [75]:
DIAS_SIN_STOCK = pd.merge(query,corridas_fechas,on='id_sc2_corrida',how='left').drop('fecha_alta_x',axis=1).rename(columns={'fecha_alta_y':'fecha_alta'})

In [ ]:
gpby = DIAS_SIN_STOCK.groupby(['id_sc2_producto','talle','color','tipo'])
#.loc['fecha_alta'].min()
gpby.loc[gpby['stock'] < 1,'INTERVALO'] = 
#loc[MERGE_FALTA_CORRIDA['falta_corrida'] > 0,'falta_corrida'] = 1


In [100]:
for index,row in corridas_fechas.iterrows():
    
    #CORRIDA ACTUAL
    corrida_actual = query[query['id_sc2_corrida'] == row['id_sc2_corrida']]
    corrida_actual.set_index(['id_sc2_producto','talle','tipo','color'], inplace=True)
    corrida_actual.reset_index(inplace=True)
    corrida_actual.fillna(0,inplace=True)
    #copio stocks NaN.
        
    STOCKS_NAN = corrida_actual[corrida_actual['stock'] == 0]
    
    #MERGE PARA MANEJAR DELTA DIAS EN UNA COLUMNA Y TENER LA FECHA DE CADA CORRIDA EN EL MISMO DF
    STOCKS_NAN_FECHA = pd.merge(STOCKS_NAN,corridas_fechas,on='id_sc2_corrida',how='left').drop('fecha_alta_x',axis=1).rename(columns={'fecha_alta_y':'fecha_alta'})
    STOCKS_NAN_FECHA.set_index(['id_sc2_producto','talle','tipo','color'], inplace=True)
    STOCKS_NAN_FECHA.reset_index(inplace=True)
#   break
    if index == 0:
        Result = STOCKS_NAN_FECHA.copy()
    else:
        Result = pd.concat([Result,STOCKS_NAN_FECHA],axis=1)    

In [108]:
query

,id_sc2_corrida,id_sc2_producto,talle,color,fecha_alta,corrida_aux,stock,precio,precio_original,tipo,origen,high_stock,falta_corrida
4056430,1,Lingerie150904001,L,Black,2019-06-12,1.0,36.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
2649269,1,Lingerie150904001,L,Sexy,NaT,NaN,NaN,NaN,NaN,None,None,0,1
485263,1,Lingerie150904001,M,Black,2019-06-12,1.0,50.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
4724532,1,Lingerie150904001,M,Sexy,NaT,NaN,NaN,NaN,NaN,None,None,0,1
514037,1,Lingerie150904001,S,Black,2019-06-12,1.0,59.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
4120736,1,Lingerie150904001,S,Sexy,NaT,NaN,NaN,NaN,NaN,None,None,0,1
4057114,1,RBLA170907201,L,Red,2019-06-12,1.0,59.0,27.0,27.0,BLAZER,SHEIN,0,0
3821844,1,RBLA170907201,M,Red,2019-06-12,1.0,49.0,27.0,27.0,BLAZER,SHEIN,0,0
4536922,1,RBLA170907201,S,Red,2019-06-12,1.0,65.0,27.0,27.0,BLAZER,SHEIN,0,0
4068268,1,RBLO160406103,L,Army Green,2019-06-12,1.0,38.0,16.0,16.0,BLUSA,SHEIN,0,0


In [98]:
query

,id_sc2_corrida,id_sc2_producto,talle,color,fecha_alta,corrida_aux,stock,precio,precio_original,tipo,origen,high_stock,falta_corrida
4056430,1,Lingerie150904001,L,Black,2019-06-12,1.0,36.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
2649269,1,Lingerie150904001,L,Sexy,NaT,NaN,NaN,NaN,NaN,None,None,0,1
485263,1,Lingerie150904001,M,Black,2019-06-12,1.0,50.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
4724532,1,Lingerie150904001,M,Sexy,NaT,NaN,NaN,NaN,NaN,None,None,0,1
514037,1,Lingerie150904001,S,Black,2019-06-12,1.0,59.0,8.0,8.0,CAMISETAS S/MANGA,SHEIN,0,0
4120736,1,Lingerie150904001,S,Sexy,NaT,NaN,NaN,NaN,NaN,None,None,0,1
4057114,1,RBLA170907201,L,Red,2019-06-12,1.0,59.0,27.0,27.0,BLAZER,SHEIN,0,0
3821844,1,RBLA170907201,M,Red,2019-06-12,1.0,49.0,27.0,27.0,BLAZER,SHEIN,0,0
4536922,1,RBLA170907201,S,Red,2019-06-12,1.0,65.0,27.0,27.0,BLAZER,SHEIN,0,0
4068268,1,RBLO160406103,L,Army Green,2019-06-12,1.0,38.0,16.0,16.0,BLUSA,SHEIN,0,0


In [90]:
STOCKS_NAN_FECHA[STOCKS_NAN_FECHA['id_sc2_producto'].isnull()]

,index,id_sc2_producto,talle,tipo,color,id_sc2_corrida,corrida_aux,stock,precio,precio_original,origen,high_stock,falta_corrida,fecha_alta,timestamp,delta_dias


In [93]:
pd.merge(STOCKS_NAN,corridas_fechas,on='id_sc2_corrida',how='left').drop('fecha_alta_x',axis=1).rename(columns={'fecha_alta_y':'fecha_alta'})

,id_sc2_producto,talle,tipo,color,id_sc2_corrida,corrida_aux,stock,precio,precio_original,origen,high_stock,falta_corrida,fecha_alta,timestamp,delta_dias
0,Lingerie150904001,L,0,Sexy,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
1,Lingerie150904001,M,0,Sexy,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
2,Lingerie150904001,S,0,Sexy,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
3,RBLO170511003,L,0,Multicolor,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
4,RBLO170511003,M,0,Multicolor,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
5,RBLO170511003,S,0,Multicolor,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
6,RBLO170602101,L,0,Pink,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
7,RBLO170602101,M,0,Pink,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
8,RBLO170602101,S,0,Pink,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3
9,RBLO170602101,XL,0,Pink,12,0.0,0.0,0.0,0.0,0,0,1,2019-08-22,3 days 00:00:00,3


In [70]:
len(STOCKS_NAN_FECHA)

247329

In [247]:
#STOCKS_NAN_FECHA

In [50]:
corridas_fechas

,fecha_alta,id_sc2_corrida,timestamp,delta_dias
0,2019-06-12,1,0,0
1,2019-06-21,2,9 days 00:00:00,9
2,2019-06-27,3,6 days 00:00:00,6
3,2019-07-01,4,4 days 00:00:00,4
4,2019-07-10,5,9 days 00:00:00,9
5,2019-07-22,6,12 days 00:00:00,12
6,2019-07-30,7,8 days 00:00:00,8
7,2019-08-05,8,6 days 00:00:00,6
8,2019-08-08,9,3 days 00:00:00,3
9,2019-08-12,10,4 days 00:00:00,4


In [18]:
# INSERTAR PRODUCTOS
server = 'tcp:192.168.1.6'
database ="planeamiento"
username = 'sa'
password = 'sa' 
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server
                      +';DATABASE='+database
                      +';UID='+username
                      +';PWD='+ password)



cursor = cnxn.cursor()
for index,row in MERGE_FALTA_CORRIDA.iterrows():
                                                                                                                   
    cursor.execute("INSERT INTO sc_indicadores([origen],[id_sc2_producto],[talle],[color],[demanda],[abastecimiento],[cantidad_abastecimiento],[desde_corrida],[hasta_corrida],[fecha_alta],[flag_corrida],[flag_stock])values (?,?,?,?,?,?,?,?,?,?,?,?)",
                                                                                                                                                                       
                   row['origen'],
                   row['id_sc2_producto'],
                   row['talle'],
                   row['color'], 
                   row['demanda'],
                   row['abastecimiento'],
                   row['contar_suba'],
                   DESDE_CORRIDA,
                   HASTA_CORRIDA,
                   f'{fecha}',
                   row['falta_corrida'],
                   row['high_stock']
                  ) 
    cnxn.commit()

cursor.close()
cnxn.close()

SystemError: <class 'pyodbc.Error'> returned a result with an error set

In [35]:
TOSQL[(TOSQL['id_sc2_producto'] =='swtee04190318400')&(TOSQL['talle']=='S')&(TOSQL['color']=='White')]

,origen,id_sc2_producto,talle,color,demanda,abastecimiento,contar_suba,desde_corrida,hasta_corrida,fecha_alta,flag_corrida,flag_stock
255455,SHEIN,swtee04190318400,S,White,162.0,124.964912,4,1,6,2019-08-09,0,0


In [34]:
query[(query['id_sc2_producto'] =='swtee04190318400')&(query['talle']=='S')&(query['color']=='White')]

,id_sc2_corrida,id_sc2_producto,talle,color,fecha_alta,corrida_aux,stock,precio,precio_original,tipo,origen,high_stock,falta_corrida
344583,1,swtee04190318400,S,White,2019-06-12,1.0,3.0,10.0,10.0,TOP,SHEIN,0,0
359067,2,swtee04190318400,S,White,2019-06-21,2.0,179.0,10.0,10.0,TOP,SHEIN,0,0
851886,3,swtee04190318400,S,White,2019-06-27,3.0,801.0,10.0,10.0,TOP,SHEIN,0,0
1132849,4,swtee04190318400,S,White,2019-07-01,4.0,153.0,10.0,10.0,TOP,SHEIN,0,0
347873,5,swtee04190318400,S,White,2019-07-10,5.0,555.0,10.0,10.0,TOP,SHEIN,0,0
971324,6,swtee04190318400,S,White,2019-07-22,6.0,646.0,10.0,10.0,TOP,SHEIN,0,0
1002050,7,swtee04190318400,S,White,2019-07-30,7.0,411.0,10.0,10.0,TOP,SHEIN,0,0
1149343,8,swtee04190318400,S,White,2019-08-05,8.0,497.0,10.0,10.0,TOP,None,0,0
839570,9,swtee04190318400,S,White,2019-08-08,9.0,456.0,10.0,10.0,TOP,None,0,0


,id_sc2_producto,talle,tipo,color,id_sc2_corrida,fecha_alta,corrida_aux,stock,precio,precio_original,...,resta_tmstp,delta_dias_resta,suma_stock,suma_tmstp,delta_dias_suma,contar_suba,demanda,abastecimiento,high_stock,falta_corrida
150216,sweatsh180828167,L,BUZO,Black,2,2019-06-21,2.0,821.0,14.0,14.0,...,0,4,4.0,9 days 00:00:00,15,2,205.5,54.149123,0,1


417065

,origen,id_sc2_producto,talle,color,demanda,abastecimiento,contar_suba,desde_corrida,hasta_corrida,fecha_alta,flag_corrida,flag_stock
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
query = '''
    select distinct id_sc2_corrida from sc2_detalle with (nolock)
'''
query = pd.read_sql(query,cnxn)

,id_sc2_corrida
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9


In [ ]:
server = 'tcp:192.168.1.6'
database = 'planeamiento'
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+
                      ';DATABASE='+database+
                      ';UID='+username+
                      ';PWD='+ password) 

In [16]:
cursor = cnxn.cursor()

In [3]:
start1=datetime.datetime.today()
cursor = cnxn.cursor()
cursor.execute('''
truncate table planeamiento..gon_tmp_stocks 

insert into planeamiento..gon_tmp_stocks
select 
    ROW_NUMBER ( ) OVER ( order by codigo, orden, cantidad , frente, columna, nivel) id,  
    ROW_NUMBER ( ) OVER ( partition by codigo order by codigo, orden, cantidad , frente, columna, nivel) nro_caja, 
    *
from ( 
    -- esta parte simula que las cosas en Gondola, son parte del deposito.
    -- si hubiera sobrante se lo resta.                   
    select 
        s.codigo, 
        10 orden, 
        '__'+isnull(a.frente,'XYZ') frente, 
        isnull(posicion % 100 ,99) columna, 
        isnull(posicion / 100, 99 ) nivel, 
        case when isnull(ss.cantidad,0) > s.pt then 0 else s.pt - isnull(ss.cantidad,0) end cantidad, 
        1 cajas   
    from sgjb_produccion..sis_artstock s  with (nolock) 
    inner join  sgjb_produccion..itm_articulo a  with (nolock) on a.codigo11 = s.codigo
    left join (
        select codigo, sum(cantidad) cantidad, sum(cajas) cajas 
        from estimaciones..depo_posiciones p  with (nolock) 
        inner join estimaciones..depo_nombres n   with (nolock)  on n.frente= p.frente and p.columna = n.columna 
        where orden = 20 
        group by codigo 
        HAVING sum(cantidad) > 0 or sum(cajas) > 0 
        ) ss on ss.codigo = s.codigo 
    where s.pt > 0 or a.posicion is not null
    
    union all 
    
    select 
        codigo,
        orden,  
        P.FRENTE, 
        P.COLUMNA,
        P.NIVEL, 
        sum(cantidad) cantidad, 
        sum(cajas) cajas 
    from estimaciones..depo_posiciones p  with (nolock) 
    inner join estimaciones..depo_nombres n   with (nolock)  on n.frente= p.frente and p.columna = n.columna 
    group by 
        codigo,
        orden,  
        P.FRENTE, 
        P.COLUMNA,
        P.NIVEL
    HAVING SUM(cajas) > 0 or sum(cantidad) > 0
 )  x 
''') 
cnxn.commit()

end1=datetime.datetime.today()

In [4]:

start2=datetime.datetime.today()

cursor.execute( '''

truncate table planeamiento..gon_tmp_horizonte

insert into planeamiento..gon_tmp_horizonte
select max(id_fecha) id_fecha   
from planeamiento..proy_inventario with (nolock) 
where idproyeccion in ( select idproyeccion from planeamiento..proy_Proyecciones with (nolock) where estado = 'A' )

''')
cnxn.commit()
end2=datetime.datetime.today()


In [19]:
start3=datetime.datetime.today()

cursor.execute( '''

truncate table planeamiento..gon_tmp_ventas
insert into planeamiento..gon_tmp_ventas     
select 
    a.codigo, 
    y.id_fecha, 
    y.fecha  , 
    -- en muy pocos casos, la dem_acumulada_anterior del id_fecha = 0 da un numero distinto de cero, y deberia ser siempre cero.
    -- asi que lo fixeo a mano.
    case when y.id_fecha = 0 then 0 else isnull(d.dem_acumulada_anterior,0) end  dem_acumulada_anterior, 
    -- el siguiente caso es para cuando los productos venden = 0 (discontinuos) o cuando tienen mucho stock.
     -- al agregarle un valor muy alto, todo lo que sobra le va a poner el ultimo id_fecha
    (case when y.id_fecha = z.id_fecha then 999999 else 0 end ) +   
        isnull(d.dem_acumulada,0) dem_acumulada
    
from (     
    select  a.codigo from planeamiento..articulos a with (nolock)
    where exists ( select top 1 1 from planeamiento..gon_tmp_stocks with (nolock) where codigo  = a.codigo )
    ) a      
cross join 
    ( 
    select id_fecha, fecha  
    from planeamiento..proy_calendario with (nolock) 
    where id_fecha >= 0 and id_fecha <= ( select id_fecha from planeamiento..gon_tmp_horizonte with (nolock) )
    and idproyeccion in ( select idproyeccion from planeamiento..proy_Proyecciones with (nolock) where estado = 'A' )  
    ) y
cross join (
    select id_fecha from planeamiento..gon_tmp_horizonte with (nolock) 
) z 
left join (
        select
            a.codigo, 
            t1.id_articulo,
           t1.id_fecha,
           t1.demanda,
           sum(t2.demanda) - t1.demanda + (case when t1.id_fecha = 0 then 0 when t1.id_fecha = 1 then 0.33 when t1.id_fecha = 2 then 0.66 else 1 end ) * isnull(re.saldo,0) dem_acumulada_anterior,
           sum(t2.demanda) + (case when t1.id_fecha = 0 then .33 when t1.id_fecha = 1 then 0.66 else 1 end ) * isnull(re.saldo,0) dem_acumulada  ,
           re.saldo
    from gon_tmp_ventas1 t1
    inner join gon_tmp_ventas1  t2 on t2.id_articulo = t1.id_articulo and t2.id_fecha <= t1.id_fecha  
   inner join planeamiento..articulos a   with (nolock) on a.idArticulos = t1.id_articulo  
   left join (   
                select reemplazante codigo, sum(saldo) saldo
                from planeamiento..pik_ordenes_reemplazos  with (nolock) 
                where estado = 'A' 
                group by reemplazante 
           ) re on re.codigo = a.codigo
    group by a.codigo,
            t1.id_articulo,
           t1.id_fecha,
           t1.demanda,
           re.saldo 
    ) d on d.codigo = a.codigo  and d.id_fecha = y.id_fecha   


''')
cnxn.commit()
end3=datetime.datetime.today()

In [18]:
print(f"3 :{str(end3-start3)}")

3 :0:01:33.540872


In [20]:
print(f"3 :{str(end3-start3)}")

3 :0:00:16.739751


In [21]:
start4=datetime.datetime.today()

cursor.execute('''

 
truncate table  planeamiento..gon_tmp_grupos

insert into planeamiento..gon_tmp_grupos 
select g.codigo, case when g.cod8grupo <> '' then g.cod8grupo+g.talle else null end codigo_grupo,  id_fecha_Grupo
from  estimaciones..grp_AsignacionDeGruposCompleto g  with (nolock)
left join ( select codigo, idFechaSO_SaldoCorte id_fecha_Grupo from estimaciones..pry_vStockOuts2  with (nolock) ) x on x.codigo = g.cod8grupo+g.talle


truncate table  planeamiento..gon_tmp_ordenes

insert into planeamiento..gon_tmp_ordenes
select desde codigo, sum(saldo) saldo
from estimaciones..grp_SaldosDePasesDesde with (nolock)
where estado = 'PE'
group by desde



''')
cnxn.commit()
end4=datetime.datetime.today()

In [22]:
print(f"4 :{str(end4-start4)}")

4 :0:01:05.355420


In [7]:
start5=datetime.datetime.today()

cursor.execute('''
truncate table planeamiento..gon_tmp_listas
insert into planeamiento..gon_tmp_listas

select
    codigo,
    d.frente,
    n.columna,
    d.nivel
     
from planeamiento..gon_DetalleListaAbastecimiento d with (nolock)
    inner join planeamiento..gon_ListaAbastecimiento l with (nolock) on l.idListaAbastecimiento = d.idListaAbastecimiento
    inner join estimaciones..depo_nombres n with (nolock) on n.frente = d.frente and n.columna = d.columna
    inner join planeamiento..gon_EstadosListaAbastecimiento e with (nolock) on e.idListaAbastecimiento = d.idListaAbastecimiento and e.estado = 'A' and e.hasta is null 
''')
cnxn.commit()
end5=datetime.datetime.today()

In [8]:
start6=datetime.datetime.today()

cursor.execute('''
 truncate table planeamiento..gon_tmp_resultado
 insert into planeamiento..gon_tmp_resultado  
select 
    z.id,
    z.nro_caja,
    z.codigo,
    z.orden,
    z.frente,
    z.columna,
    z.nivel,
    z.cantidad,
    z.cajas,
    z.cantidad_acum,
    v.id_fecha,
    v.fecha,
    v.dem_acumulada_anterior,
    v.dem_acumulada,
    g.codigo_grupo,
    g.id_fecha_grupo,
    case when l.codigo is null then 0 else 1 end en_lista_activa,
    getdate() __ts,
    aa.posicion,
    case when o.codigo is not null then 1 else 0 end requerido_orden     
from ( 
    select 
        t1.id,
        t1.nro_caja,
        t1.codigo,
        t1.orden,
        t1.frente,
        t1.columna,
        t1.nivel,
        t1.cantidad,
        t1.cajas,
        sum(t2.cantidad) cantidad_acum
    from planeamiento..gon_tmp_stocks t1 with (nolock)
    inner join planeamiento..gon_tmp_stocks t2 with (nolock) on t1.codigo = t2.codigo and t1.id >= t2.id
    group by 
        t1.id,
        t1.nro_caja,
        t1.codigo,
        t1.orden,
        t1.frente,
        t1.columna,
        t1.nivel,
        t1.cantidad,
        t1.cajas
    ) z 
inner join planeamiento..gon_tmp_ventas v  with (nolock) on v.codigo = z.codigo and z.cantidad_acum - z.cantidad >= v.dem_acumulada_anterior and z.cantidad_acum - z.cantidad < v.dem_acumulada
inner join planeamiento..articulos aa with (nolock) on aa.codigo = z.codigo
left join planeamiento..gon_tmp_listas l  with (nolock) on l.codigo = z.codigo and l.frente =  z.frente and l.columna  =z.columna and l.nivel = z.nivel 
left join planeamiento..gon_tmp_grupos g  with (nolock) on g.codigo = z.codigo
left join planeamiento..gon_tmp_ordenes o with (nolock) on o.codigo = z.codigo 



''')
cnxn.commit()
end6=datetime.datetime.today()

In [9]:
print(f"1 :{str(end1-start1)}")
print(f"2 :{str(end2-start2)}")
print(f"3 :{str(end3-start3)}")
print(f"4 :{str(end4-start4)}")
print(f"5 :{str(end5-start5)}")
print(f"6 :{str(end6-start6)}")

1 :0:00:02.201675
2 :0:00:00.338655
3 :0:01:07.852616
4 :0:00:48.147237
5 :0:00:00.255117
6 :0:00:04.922075


In [181]:
cnxn.close()

datetime.datetime(2019, 8, 27, 15, 1, 55, 841778)

In [27]:
start4a=datetime.datetime.today()

cursor.execute('''
 
truncate table  planeamiento..gon_tmp_grupos

insert into planeamiento..gon_tmp_grupos 
select g.codigo, case when g.cod8grupo <> '' then g.cod8grupo+g.talle else null end codigo_grupo,  id_fecha_Grupo
from  estimaciones..grp_AsignacionDeGruposCompleto g  with (nolock)
left join ( select codigo, idFechaSO_SaldoCorte id_fecha_Grupo from estimaciones..pry_vStockOuts2  with (nolock) ) x on x.codigo = g.cod8grupo+g.talle
''')
cnxn.commit()
end4a=datetime.datetime.today()

In [28]:
print(f"4 :{str(end4a-start4a)}")
# 1min antes
# 0:01:00.132816

4 :0:01:00.132816


In [25]:
start4b=datetime.datetime.today()

cursor.execute('''

truncate table  planeamiento..gon_tmp_ordenes

insert into planeamiento..gon_tmp_ordenes
select desde codigo, sum(saldo) saldo
from estimaciones..grp_SaldosDePasesDesde with (nolock)
where estado = 'PE'
group by desde

''')
cnxn.commit()
end4b=datetime.datetime.today()

In [26]:
print(f"4 :{str(end4b-start4b)}")

4 :0:00:00.634034


74

In [264]:
import os
os.getcwd()

'/home/aa/cloudJ/JB/PRD/scrapper/Scripts'